In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [0]:
df = pd.read_csv('train_FD001.txt', sep = ' ', header = None)
df = df.drop(columns=[26, 27])
df.columns = (["n_engine","cycle","opset_1","opset_2","opset_3",
              "sens_1","sens_2","sens_3","sens_4","sens_5","sens_6","sens_7","sens_8","sens_9","sens_10",
              "sens_11","sens_12","sens_13","sens_14","sens_15","sens_16","sens_17","sens_18","sens_19","sens_20","sens_21"])

In [0]:
df = df.drop(['sens_1','sens_5','sens_6','sens_10','sens_16','sens_18','sens_19'],axis=1)

In [0]:
#Agregada variable RUL

RUL_temp=np.zeros(0)
for i in range(1,df['n_engine'].max() + 1):
    minus = np.linspace(1,df[df['n_engine']==i]['cycle'].max(),df[df['n_engine']==i]['cycle'].max())
    RUL_engine = np.ones((df[df['n_engine']==i]['cycle'].max()))*df[df['n_engine']==i]['cycle'].max()
    RUL_temp = np.append(RUL_temp,RUL_engine-minus)
df['RUL'] = RUL_temp
df['RUL'] = df['RUL'].astype(int)

In [5]:
df.head()

,n_engine,cycle,opset_1,opset_2,opset_3,sens_2,sens_3,sens_4,sens_7,sens_8,sens_9,sens_11,sens_12,sens_13,sens_14,sens_15,sens_17,sens_20,sens_21,RUL
0,1,1,-0.0007,-0.0004,100.0,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187


In [0]:
features_maps=np.empty((15,14,1))
df_ys = np.zeros((1,1))
for i in range(1,df['n_engine'].max()+1):
  #se obtienen los dataset de engines independientes en df_temp
  df_temp1 = df[df['n_engine'] == i]
  df_temp2 = df_temp1.drop(['n_engine','cycle','opset_1','opset_2','opset_3'],axis=1)
  df_features = df_temp2.drop(['RUL'],axis = 1)
  df_y = df_temp2['RUL'].to_numpy()
  df_ys = np.append(df_ys,df_y[14:])
  scaler = MinMaxScaler(feature_range=(-1,1))
  #minmaxsclaler aplicado entre (-1,1)
  df_features = scaler.fit_transform(df_features)
  #minmaxscaler
  for j in range(len(df_y)-14):
    feature_map = df_features[i:i+15][:]
    features_maps = np.dstack((features_maps, feature_map))
features_maps=np.delete(features_maps, 0, 2)
df_ys = np.delete(df_ys, 0, 0)

In [7]:
#como acceder a un feature map
print(features_maps.shape)
print(feature_map.shape)

(15, 14, 19231)
(15, 14)


In [8]:
print(df_ys.shape)
print(df_y.shape)
features_maps = np.moveaxis(features_maps, 2, 0)
print(features_maps.shape)

(19231,)
(200,)
(19231, 15, 14)


In [9]:
df_ys = np.expand_dims(df_ys,axis=1)
features_maps= np.expand_dims(features_maps,3)
print(df_ys.shape)
print(features_maps.shape)

(19231, 1)
(19231, 15, 14, 1)


# CNN 

Propuesta de Li, Ding, Sun

![Imagen](https://github.com/farayal/memoria_turbofan/blob/master/cnn_regression.png?raw=true)

In [10]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

import keras.backend as K
K.set_image_data_format('channels_last')


Using TensorFlow backend.


In [0]:
def cnnturbofan(input_shape):
   
    X_input = Input(input_shape)

    #Layer 1
    # Zero-Padding: Para originar un output (15,17) en la siguiente capa
    X = ZeroPadding2D((7, 0))(X_input)

    # CONV -> TANH aplicado a X
    X = Conv2D(10, (10, 1), strides = (1, 1), name = 'conv0')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('tanh')(X)

    #Layer 2
    X = ZeroPadding2D((7, 0))(X_input)
    X = Conv2D(10, (10, 1), strides = (1, 1), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('tanh')(X)

    #Layer 3
    X = ZeroPadding2D((7, 0))(X_input)
    X = Conv2D(10, (10, 1), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('tanh')(X)

    #Layer 4
    X = ZeroPadding2D((7, 0))(X_input)
    X = Conv2D(10, (10, 1), strides = (1, 1), name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('tanh')(X)

    #Layer 5
    X = ZeroPadding2D((2, 0))(X_input)
    X = Conv2D(1, (3, 1), strides = (1, 1), name = 'conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('tanh')(X)

    # FLATTEN X ("desenrollamos el feature map anterior") + FULLYCONNECTED
    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(100, activation='tanh', name='fully-connected')(X)
    X = Dense(1,activation='linear',name = 'rul-neuron')(X)

    # instancia modelo final
    model = Model(inputs = X_input, outputs = X, name='cnnturbofan')
    
  
    return model

In [12]:
cnnturbofan = cnnturbofan((features_maps.shape[1],features_maps.shape[2],features_maps.shape[3]))












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
cnnturbofan.compile(optimizer = "Adam", loss = "mean_squared_error", metrics = ["mse","accuracy"])

In [14]:
cnnturbofan.fit(x = features_maps, y = df_ys , epochs = 250, batch_size = 512)



Epoch 1/250
19231/19231 [==============================] - 4s 190us/step - loss: 12768.6811 - mean_squared_error: 12768.6811 - acc: 0.0057
Epoch 2/250
19231/19231 [==============================] - 0s 17us/step - loss: 11323.5347 - mean_squared_error: 11323.5347 - acc: 0.0046
Epoch 3/250
19231/19231 [==============================] - 0s 16us/step - loss: 10614.6627 - mean_squared_error: 10614.6627 - acc: 0.0049
Epoch 4/250
19231/19231 [==============================] - 0s 17us/step - loss: 9998.0397 - mean_squared_error: 9998.0397 - acc: 0.0044
Epoch 5/250
19231/19231 [==============================] - 0s 16us/step - loss: 9439.1918 - mean_squared_error: 9439.1918 - acc: 0.0064
Epoch 6/250
19231/19231 [==============================] - 0s 17us/step - loss: 8928.8135 - mean_squared_error: 8928.8135 - acc: 0.0047
Epoch 7/250
19231/19231 [==============================] - 0s 18us/step - loss: 8462.8822 - mean_squared_error: 8462.8822 - acc: 0.0048
Epoch 8/250
19231/19231 [==============

In [15]:
cnnturbofan.summary()

Model: "cnnturbofan"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15, 14, 1)         0         
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 19, 14, 1)         0         
_________________________________________________________________
conv4 (Conv2D)               (None, 17, 14, 1)         4         
_________________________________________________________________
bn4 (BatchNormalization)     (None, 17, 14, 1)         4         
_________________________________________________________________
activation_5 (Activation)    (None, 17, 14, 1)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 238)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 238)               

In [16]:
cnnturbofan.predict(features_maps)

array([[88.91917],
       [88.91917],
       [88.91917],
       ...,
       [94.35182],
       [94.35182],
       [94.35182]], dtype=float32)